# 第15回 機械学習自動化技術（AutoML）に触れてみよう！

___
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/kimusaku/begin-python-2023/blob/main/workbook/lecture15.ipynb)

___

## この授業で学ぶこと

最終回の今回は、14回で学んだ機械学習を自動化する技術であるAutoMLがどのようなものかを知るため、Fujitsu AutoMLの無料版を使って機械学習モデルの自動構築を体験してもらう。

Fujitsu AutoMLの概要はこちらの[リンク](https://automl.jp.fujitsu.com/ja/)を参照のこと。

## 利用手順

まずはFujitsu AutoMLのライブラリをインストールする必要がある。 `pip install`でインストールすることが出来る。

今回も、前回利用してきたダイヤモンドのデータを利用する。

In [1]:
pip install fujitsu-automl

In [2]:
import pandas as pd
from sapientml import SapientML
from sapientml.util.logging import setup_logger
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import seaborn as sns
import os

# ToyoNet-ACEで公開しているアカウント情報を貼り付ける。2024/07/17まで有効

In [3]:
train_data = sns.load_dataset('diamonds')
train_data

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75
...,...,...,...,...,...,...,...,...,...,...
53935,0.72,Ideal,D,SI1,60.8,57.0,2757,5.75,5.76,3.50
53936,0.72,Good,D,SI1,63.1,55.0,2757,5.69,5.75,3.61
53937,0.70,Very Good,D,SI1,62.8,60.0,2757,5.66,5.68,3.56
53938,0.86,Premium,H,SI2,61.0,58.0,2757,6.15,6.12,3.74


Fujitsu AutoMLでは、データセットと目的変数を指定することでその目的変数を予測するのに適している機械学習モデルの種類と前処理を自動で選択し、プログラムを組み立てて学習・推論を実行することができる。

以下のコードセルでは、ダイヤモンドのデータの `cut` 列を予測するモデルを構築する。

In [4]:
train_data, test_data = train_test_split(train_data)

y_true = test_data["cut"].reset_index(drop=True)
test_data.drop(["cut"], axis=1, inplace=True)

cls = SapientML(["cut"], model_type="fujitsu-automl")
setup_logger().handlers.clear()
cls.fit(train_data)

INFO:sapientml:Loading dataset...
INFO:sapientml:Generating pipelines...
INFO:sapientml:Generating meta features...
INFO:sapientml:Calling WebAPIs for generating pipelines...
INFO:sapientml:Authenticating app via OAuth 2.0 Client Credentials Flow
INFO:sapientml:Experiment Id: f9afa7fb-2f4d-4bed-9c71-60faa1c1e8a7
INFO:sapientml:Executing generated pipelines...
INFO:sapientml:Running script (1/3)...
INFO:sapientml:Running script (2/3)...
INFO:sapientml:Running script (3/3)...
INFO:sapientml:Evaluating execution results of generated pipelines...
INFO:sapientml:Building model by generated pipeline...
INFO:sapientml:Done.


構築したモデルで予測を行うには以下のコードセルを実行する。

In [5]:
y_pred = cls.predict(test_data)

print("Accuracy:", accuracy_score(y_true, y_pred))

INFO:sapientml:Predicting by built model...


Accuracy: 0.757285873192436


モデルの構築・予測のために生成されたプログラムは以下で確認することができる。

In [6]:
train_script = cls.model.files["final_train.py"].decode("utf-8")
print(train_script)

# *** GENERATED PIPELINE ***

# LOAD DATA
import pandas as pd
train_dataset = pd.read_pickle("./training.pkl")

import pickle


# PREPROCESSING-1
import numpy as np
NUMERIC_COLS_TO_SCALE = ['carat', 'depth', 'table', 'price']
train_dataset[NUMERIC_COLS_TO_SCALE] = np.log1p(train_dataset[NUMERIC_COLS_TO_SCALE])

# DETACH TARGET
TARGET_COLUMNS = ['cut']
feature_train = train_dataset.drop(TARGET_COLUMNS, axis=1)
target_train = train_dataset[TARGET_COLUMNS].copy()

# PREPROCESSING-2
from sklearn.preprocessing import OneHotEncoder
CATEGORICAL_COLS = ['color', 'clarity']
onehot_encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
train_encoded = pd.DataFrame(onehot_encoder.fit_transform(feature_train[CATEGORICAL_COLS]), columns=onehot_encoder.get_feature_names_out(), index=feature_train.index)
feature_train = pd.concat([feature_train, train_encoded ], axis=1)
feature_train.drop(CATEGORICAL_COLS, axis=1, inplace=True)
with open('oneHotEncoder.pkl', 'wb') as f:
    pickle.dump

In [7]:
predict_script = cls.model.files["final_predict.py"].decode("utf-8")
print(predict_script)

# *** GENERATED PIPELINE ***

# LOAD DATA
import pandas as pd
test_dataset = pd.read_pickle("./test.pkl")

import pickle


# PREPROCESSING-1
import numpy as np
NUMERIC_COLS_TO_SCALE = ['carat', 'depth', 'table', 'price']
NUMERIC_COLS_TO_SCALE_FOR_TEST = list(set(test_dataset.columns) & set(NUMERIC_COLS_TO_SCALE))
test_dataset[NUMERIC_COLS_TO_SCALE_FOR_TEST] = np.log1p(test_dataset[NUMERIC_COLS_TO_SCALE_FOR_TEST])

# DETACH TARGET
TARGET_COLUMNS = ['cut']
if set(TARGET_COLUMNS).issubset(test_dataset.columns.tolist()):
    feature_test = test_dataset.drop(TARGET_COLUMNS, axis=1)
    target_test = test_dataset[TARGET_COLUMNS].copy()
else:
    feature_test = test_dataset

# PREPROCESSING-2
with open('oneHotEncoder.pkl', 'rb') as f:
    onehot_encoder = pickle.load(f)
CATEGORICAL_COLS = ['color', 'clarity']
test_encoded = pd.DataFrame(onehot_encoder.transform(feature_test[CATEGORICAL_COLS]), columns=onehot_encoder.get_feature_names_out(), index=feature_test.index)
feature_test = pd.concat([

## おわりに

プログラミング実習講義3,4では、Pythonによるプログラミングの基礎、基本的なアルゴリズムやデータ構造、また応用としてデータサイエンスおよび機械学習、AutoMLについて学んできた。過去に蓄積されてきたおびただしい量の便利なライブラリ・ツールに加えて近年では生成AIやノーコード・ローコード開発等の台頭により、人が必ずしも全てのコードを書かずとも高度なソフトウェアを開発することが出来ることになった。しかしながら、それらを正しく使いこなすにはプログラミングの素養がまだまだ重要である。経営学部の学生の皆様におかれても、この講義で学んだことをきっかけにプログラミングの素養を今後も引き続き磨いていってもらえることを願う。

## 演習

**課題**  
Fujitsu AutoMLを用いて、インターネットで入手可能な適当なデータセット（CSVファイル）および適当な目的変数を設定して機械学習モデルを構築し、学習・予測のために生成されたプログラムを確認しなさい。

以下のコードセルでは、URLで直接ダウンロード可能なデータセットを例として記載している。ここに新たなURLを指定するか、Google DriveにCSVファイルを置いて新しいデータを読み込んで試してみよう。Google Driveに置く方法で実施した場合は、レポートにCSVファイルも一緒に添付して提出してほしい。 

なお、データセット・目的変数の選び方によってはFujitsu AutoMLのバグに起因するエラーが発生する可能性がある。データセット・目的変数の選び方が妥当かつどうしてもエラーが解消しない場合は、エラーのまま提出してもらって構わない。バグの報告は開発者にとってはとてもありがたいものである。

In [ ]:
train_data = pd.read_csv("https://github.com/sapientml/sapientml/files/12481088/titanic.csv")
train_data

In [ ]:
train_data, test_data = train_test_split(train_data)

y_true = test_data["survived"].reset_index(drop=True)
test_data.drop(["survived"], axis=1, inplace=True)

cls = SapientML(["survived"], model_type="fujitsu-automl")
setup_logger().handlers.clear()
cls.fit(train_data)

In [ ]:
train_script = cls.model.files["final_train.py"].decode("utf-8")
print(train_script)

In [ ]:
predict_script = cls.model.files["final_predict.py"].decode("utf-8")
print(predict_script)